# --- Day 8: Treetop Tree House ---

The expedition comes across a peculiar patch of tall trees all planted carefully in a grid. The Elves explain that a previous expedition planted these trees as a reforestation effort. Now, they're curious if this would be a good location for a tree house.

First, determine whether there is enough tree cover here to keep a tree house hidden. To do this, you need to count the number of trees that are visible from outside the grid when looking directly along a row or column.

The Elves have already launched a quadcopter to generate a map with the height of each tree (your puzzle input). For example:

    30373
    25512
    65332
    33549
    35390

Each tree is represented as a single digit whose value is its height, where 0 is the shortest and 9 is the tallest.

A tree is visible if all of the other trees between it and an edge of the grid are shorter than it. Only consider trees in the same row or column; that is, only look up, down, left, or right from any given tree.

All of the trees around the edge of the grid are visible - since they are already on the edge, there are no trees to block the view. In this example, that only leaves the interior nine trees to consider:

    The top-left 5 is visible from the left and top. (It isn't visible from the right or bottom since other trees of height 5 are in the way.)
    The top-middle 5 is visible from the top and right.
    The top-right 1 is not visible from any direction; for it to be visible, there would need to only be trees of height 0 between it and an edge.
    The left-middle 5 is visible, but only from the right.
    The center 3 is not visible from any direction; for it to be visible, there would need to be only trees of at most height 2 between it and an edge.
    The right-middle 3 is visible from the right.
    In the bottom row, the middle 5 is visible, but the 3 and 4 are not.

With 16 trees visible on the edge and another 5 visible in the interior, a total of 21 trees are visible in this arrangement.

Consider your map; how many trees are visible from outside the grid?


In [134]:
with open('in.txt') as f:
	trees = f.read().splitlines()

In [135]:
with open('test.txt') as f:
	test_trees = f.read().splitlines()

In [136]:
def leftview(i, row_trees, visibles):
	if len(row_trees) == 0:		# First tree is the tallest in previous iteration
		return visibles
	elif len(row_trees) == 1:	# 2nd tree is the tallest in previous iteration
		visibles.add((i,0))
		return visibles
	else:
		tallest_position = row_trees.find(max(row_trees))
		visibles.add((i,tallest_position))
		new_row_trees = row_trees[:tallest_position]
		return leftview(i, new_row_trees, visibles)

In [137]:
def rightview_reversed(i, row_trees, visibles, og_size):
	if len(row_trees) == 0:		# First (from the right) tree is the tallest in the LAST iteration
		return visibles
	elif len(row_trees) == 1:	# First (from the right) tree is the tallest in THIS iteration
		visibles.add((i,og_size-1)) 	# -1 is the last position in the row
		return visibles
	else:
		tallest_position = row_trees.rfind(max(row_trees))
		visibles.add((i,og_size - 1 - tallest_position))
		new_row_trees = row_trees[tallest_position+1:]
		return rightview_reversed(i, new_row_trees, visibles, og_size)

In [138]:
def topview(j, col_trees, visibles):
	if len(col_trees) == 0:		# First (from the top) tree is the tallest in the LAST iteration
		return visibles
	elif len(col_trees) == 1:	# First (from the top) tree is the tallest in THIS iteration
		visibles.add((0,j))
		return visibles
	else:
		tallest_position = col_trees.find(max(col_trees))
		visibles.add((tallest_position,j))
		new_col_trees = col_trees[:tallest_position]
		return topview(j, new_col_trees, visibles)

In [139]:
def bottomview_reversed(j, col_trees, visibles, og_size):
	if len(col_trees) == 0:		# First (from the bottom) tree is the tallest in the LAST iteration
		return visibles
	elif len(col_trees) == 1:	# First (from the bottom) tree is the tallest in THIS iteration
		visibles.add((og_size-1,j))
		return visibles
	else:
		tallest_position = col_trees.rfind(max(col_trees))
		visibles.add(( og_size - 1 - tallest_position,j))
		new_col_trees = col_trees[tallest_position+1:]
		return bottomview_reversed(j, new_col_trees, visibles, og_size)

In [140]:
def transpose(trees):
	trees_transpose = [[] for i in range(len(trees[0]))]
	trees = list(zip(*trees))
	for i in range(len(trees)):
		line = ''
		for j in range(len(trees[i])):
			line += trees[i][j]
		trees_transpose[i] = line
	return trees_transpose

In [141]:
def transpose2(trees):
	tress = list(map(list, zip(*trees)))
	for i in range(len(tress)):
		tress[i] = str(tress[i]).replace(',','').replace(' ','').replace("'",'').replace('[','').replace(']','')
	return tress

In [142]:
def relfect(trees):
	for i in range(len(trees)):
		trees[i] = trees[i][::-1]
	return trees

In [143]:
visibles = set()
og = len(trees)
og_tranpose = len(trees[0])

for i, row_trees in enumerate(trees):
	visibles = leftview(i, row_trees, visibles)

trees = relfect(trees)
for i, row_trees in enumerate(trees):
	visibles = rightview_reversed(i, row_trees, visibles, og)

trees = relfect(trees)
trees_transposed = transpose(trees)
for j, col_trees in enumerate(trees_transposed):
	visibles = topview(j, col_trees, visibles)

trees_transposed = relfect(trees_transposed)
for j, col_trees in enumerate(trees_transposed):
	visibles = bottomview_reversed(j, col_trees, visibles, og_tranpose)

print(len(visibles))

1750
0
